# Solving LP problems with Xpress library

https://www.msi-jp.com/xpress/learning/square/01-python-interface.pdf

Andrea Gasparin: andrea.gasparin@PHD.units.it

In [2]:
import xpress as xp
import pandas as pd
import numpy as np

## Loading and setting data as Pandas dataframe

In [3]:
params = pd.read_csv("data1.csv")

Access info and data

In [4]:
print(params.columns) # name of the dataframe (df) columns, returned as a list
print(params["value"]) # get value column
print(params.value) # alternative way to get value column

Index(['param', 'origin', 'destination', 'type', 'value'], dtype='object')
0        55
1        12
2        80
3        13
4       115
       ... 
891    3800
892    2300
893    3800
894    2300
895    3800
Name: value, Length: 896, dtype: int64
0        55
1        12
2        80
3        13
4       115
       ... 
891    3800
892    2300
893    3800
894    2300
895    3800
Name: value, Length: 896, dtype: int64


Get sub-dataframe by key

In [5]:
sub_df = params[(params.origin == "SIN") & (params.destination == "RTM")]
print(sub_df)

    param origin destination  type  value
0      DU    SIN         RTM  DC20     55
1      DU    SIN         RTM  RC20     12
56     DU    SIN         RTM  DC40     35
57     DU    SIN         RTM  RC40      3
112    VC    SIN         RTM  DC20    406
113    VC    SIN         RTM  RC20    630
168    VC    SIN         RTM  DC40    515
169    VC    SIN         RTM  RC40    779
224    DL    SIN         RTM  DC20     25
225    DL    SIN         RTM  RC20      0
280    DL    SIN         RTM  DC40      3
281    DL    SIN         RTM  RC40      0
336     F    SIN         RTM  DC20    100
337     F    SIN         RTM  RC20    100
448     F    SIN         RTM  DC40    100
449     F    SIN         RTM  RC40    100
560    EC    SIN         RTM  DC20    283
561    EC    SIN         RTM  RC20    283
616    EC    SIN         RTM  DC40    365
617    EC    SIN         RTM  RC40    365
672    WF    SIN         RTM  DC20     19
673    WF    SIN         RTM  RC20     19
728    WF    SIN         RTM  DC40

Get single item by key

In [6]:
print(params[(params.param == "FR") & (params.origin == "SIN") & (params.destination == "RTM")].value.iloc[0])

1050


Setting all parameters from data

In [7]:
FR = params[params["param"] == "FR"].drop(columns="param")
VC = params[params["param"] == "VC"].drop(columns="param")
EC = params[params["param"] == "EC"].drop(columns="param")

N = range(params.origin.unique().shape[0])
M = range(params.destination.unique().shape[0])
K = range(params.type.unique().shape[0])

MC_ = pd.merge(FR, VC, on=["origin", "destination", "type"])
MC_["value"] = MC_.value_x - MC_.value_y
MC = MC_[["origin", "destination", "type", "value"]]

F = params[params["param"] == "F"].drop(columns="param")
IF = F.copy()

if_value = []
for row in F.iterrows():
    line = row[1]
    origin, destination, type_, f_ijk = line.origin, line.destination, line.type, line.value
    f_jik = F[(F.origin == destination) & (F.destination == origin) & (F.type == type_)].value.iloc[0]

    value = (f_ijk - f_jik) / f_ijk if f_ijk > f_jik else 0
    if_value.append(value)

IF.value = if_value

CP = 3350  # The operational capacity of the vessel (unit: TEU)
DW = 36510  # The dead weight tonnage of the vessel (unit: ton)

WF = params[params["param"] == "WF"].drop(columns="param")
WE = pd.read_csv("data_we.csv")

RF = 200  # The maximum reefer plug number of the vessel
FE = 1135  # The maximum number of 40' containers loaded by the vessel

DL = params[params["param"] == "DL"].drop(columns="param")
DU = params[params["param"] == "DU"].drop(columns="param")
CI = pd.read_csv("data_ci.csv")

Setting dicts to retrieve indexes by key

In [8]:
origins = DU.origin.unique()
or_idx = dict(zip(origins, range(len(origins))))
idx_to_or = dict(zip(range(len(origins)), origins))

destinations = DU.destination.unique()
des_idx = dict(zip(destinations, range(len(destinations))))
idx_to_des = dict(zip(range(len(destinations)), destinations))

cargo_types = DU.type.unique()
type_idx = dict(zip(cargo_types, range(len(cargo_types))))
idx_to_type = dict(zip(range(len(cargo_types)), cargo_types))

Example of retrieve element by key

In [11]:
print(DU)
for i in origins:
    for j in destinations:
        for k in cargo_types:
            print(DU[(DU.origin == i) & (DU.destination == j) & (DU.type == k)].value.iloc[0])

    origin destination  type  value
0      SIN         RTM  DC20     55
1      SIN         RTM  RC20     12
2      SIN         FXT  DC20     80
3      SIN         FXT  RC20     13
4      SIN         BRV  DC20    115
..     ...         ...   ...    ...
107    KHH         FXT  RC40      6
108    KHH         BRV  DC40     50
109    KHH         BRV  RC40      6
110    KHH         LEH  DC40     25
111    KHH         LEH  RC40      6

[112 rows x 4 columns]
55
12
35
3
80
13
29
3
115
13
36
3
25
12
20
3
45
9
41
9
55
8
50
10
75
6
70
9
10
9
18
8
20
12
27
5
24
9
29
2
47
8
45
1
15
8
19
2
10
8
29
5
15
8
26
5
19
10
31
5
10
8
23
5
18
6
37
5
21
5
25
5
30
6
36
5
11
4
20
5
15
8
29
5
19
4
26
5
17
2
30
5
15
5
30
5
25
6
45
6
41
5
28
6
60
8
50
6
13
7
25
6


Use the dict to operate on the matrix

In [13]:
y = np.array([[[k*j*i for k in K] for j in N] for i in M])
print(y.shape)

sum([y[or_idx[i], des_idx[j], type_idx[k]] for i in origins for j in destinations for k in cargo_types])

(11, 11, 4)


756